In [2]:
import os
import pymssql
import datetime
import math

import pandas as pd

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
def get_connection():
    """Establish a connection to the SQL Server database.
    Returns:
        pymssql.Connection: A connection object to the SQL Server database.
    """
    return pymssql.connect(
        server=os.getenv('SQL_SERVER_HOST'),
        user=os.getenv('SQL_SERVER_USER'),
        password=os.getenv('SQL_SERVER_PASS'),
        database='processo'
    )

In [5]:
numero_processo = '018129'
ano_processo = '2017'

query = f"""
    SELECT p.idprocesso as id_processo, 
    NumeroProcesso as numero_processo, 
    AnoProcesso as ano_processo, 
    IdComposicaoPauta as id_composicao_pauta, 
    idVotoPauta as id_voto_pauta,
    p.Assunto as assunto, 
    numero_sessao, 
    ano_sessao, 
    DataSessao as data_sessao,  
    Relatorio as relatorio, 
    FundamentacaoVoto as fundamentacao_voto, 
    Conclusao as conclusao, 
    texto_acordao, 
    o.nome as orgao_responsavel, 
    o.IdOrgao as id_orgao_responsavel,
    gp.Nome as nome_responsavel, 
    gp.Documento as documento_responsavel, 
    gp.TipoPessoa as tipo_responsavel, 
    gp.idpessoa as id_pessoa
    FROM processo.dbo.vw_ia_votos_acordaos_decisoes ia 
    LEFT JOIN processo.dbo.processos p ON ia.NumeroProcesso = p.numero_processo  AND ia.AnoProcesso = p.ano_processo 
    LEFT JOIN processo.dbo.Orgaos o ON o.IdOrgao = p.IdOrgaoEnvolvido
    LEFT JOIN processo.dbo.Pro_ProcessosResponsavelDespesa pprd ON p.IdProcesso = pprd.IdProcesso 
    LEFT JOIN processo.dbo.GenPessoa gp ON pprd.IdPessoa = gp.IdPessoa 
    WHERE p.numero_processo = {numero_processo}
    AND p.ano_processo = {ano_processo}
    """

In [6]:
df = pd.read_sql(query, get_connection())

group_cols = [
        'id_processo', 'numero_processo', 'ano_processo', 'id_composicao_pauta', 'assunto', 
        'id_voto_pauta', 'numero_sessao', 'ano_sessao', 'data_sessao', 'relatorio',
        'fundamentacao_voto', 'conclusao', 'texto_acordao', 'orgao_responsavel', 'id_orgao_responsavel',
]

person_cols = ['nome_responsavel', 'documento_responsavel', 'tipo_responsavel', 'id_pessoa']

# Agrupa o DataFrame e aplica uma função lambda para criar a lista de dicionários

df = df.groupby(group_cols, dropna=False).apply(
lambda x: pd.Series({'responsaveis': x[person_cols].apply(
        lambda y: y.dropna().to_dict(), axis=1
).tolist()})).reset_index()


C:\Users\05911205424\AppData\Local\Temp\ipykernel_5704\1959158049.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())
C:\Users\05911205424\AppData\Local\Temp\ipykernel_5704\1959158049.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(group_cols, dropna=False).apply(


In [7]:
df

,id_processo,numero_processo,ano_processo,id_composicao_pauta,assunto,id_voto_pauta,numero_sessao,ano_sessao,data_sessao,relatorio,fundamentacao_voto,conclusao,texto_acordao,orgao_responsavel,id_orgao_responsavel,responsaveis
0,463315,018129,2017,112913,APRECIAÇÃO DE CONCESSÃO DE APOSENTADORIA ...,36117,00089,2023,2023-12-12 00:00:00,I - RELATÓRIO\r\n\r\nVersam os autos acerca da...,II – FUNDAMENTAÇÃO\r\n\r\nA matéria sob exame ...,"III – CONCLUSÃO\r\n\r\nAnte o exposto, em harm...",DECIDEM os Conselheiros do Tribunal de Contas ...,SECRETARIA DE ESTADO DA SAÚDE PUBLICA,521,[{'nome_responsavel': 'RAQUEL VICENTE DA SILVA...
1,463315,018129,2017,120725,APRECIAÇÃO DE CONCESSÃO DE APOSENTADORIA ...,43982,0013V,2025,2025-07-14 10:00:00,"I RELATÓRIO.\r\n\r\n\r\n\r\nEm exame, para fin...","II. FUNDAMENTAÇÃO\r\n\r\n\r\n\r\nInicialmente,...","III DISPOSITIVO.\r\n\r\n\r\n\r\nNesses termos,...","Vistos, relatados e discutidos estes autos, AC...",SECRETARIA DE ESTADO DA SAÚDE PUBLICA,521,[{'nome_responsavel': 'RAQUEL VICENTE DA SILVA...


In [9]:
id_processo = df.iloc[0].id_processo

In [10]:
id_processo

np.int64(463315)

In [11]:
from models import (
    ObrigacaoORM, 
    RecomendacaoORM, 
    CancelamentoObrigacao, 
    CancelamentoRecomendacao, 
    engine_dip,
    get_db_dip,
    BaseDIP
)

In [15]:
db_dip = next(get_db_dip())

In [17]:
db_dip.query(ObrigacaoORM).filter(ObrigacaoORM.IdProcesso == int(id_processo)).all()